In [84]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import seaborn as sns
import matplotlib as mpl
from datetime import datetime
import plotly.express as px
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [85]:
ls'/content/gdrive/My Drive/Thesis'

dataframe_dma_flow_1D.csv     dataframe_flow_variable_1D.csv
dataframe_dma_flow_60min.csv  dataframe_flow_variable_60min.csv
dataframe_flow_1D.csv


In [86]:
data = pd.read_csv("/content/gdrive/My Drive/Thesis/dataframe_flow_variable_60min.csv")

In [87]:
data['Unnamed: 0'] = pd.to_datetime(data['Unnamed: 0'])

In [88]:
data.rename( columns={'Unnamed: 0':'Date_time','225':'DMA_225'}, inplace=True )

In [89]:
data_DMA225 = data[['Date_time','DMA_225']]

In [ ]:
#data_DMA225 = pd.read_csv("/content/gdrive/My Drive/Thesis/dataframe_flow_variable_60min.csv",usecols=[1])

In [ ]:
#data_DMA225.rename( columns={'225':'DMA_225'}, inplace=True )

In [90]:
data_DMA225

,Date_time,DMA_225
0,2016-04-22 00:00:00,1.13875
1,2016-04-22 01:00:00,1.16650
2,2016-04-22 02:00:00,1.16650
3,2016-04-22 03:00:00,1.33300
4,2016-04-22 04:00:00,1.33300
...,...,...
2203,2016-07-22 19:00:00,3.97225
2204,2016-07-22 20:00:00,3.41650
2205,2016-07-22 21:00:00,2.88900
2206,2016-07-22 22:00:00,2.22200


In [91]:
data_DMA225.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2208 entries, 0 to 2207
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Date_time  2208 non-null   datetime64[ns]
 1   DMA_225    2208 non-null   float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 34.6 KB


In [92]:
# Train Test Split Index
size_of_trainset = 0.7
split_value = round(len(data_DMA225)* size_of_trainset)
print (split_value)

# Split
train = data_DMA225.iloc[:split_value]
test = data_DMA225.iloc[split_value:]

print(data_DMA225.shape)
print(train.shape,test.shape)

1546
(2208, 2)
(1546, 2) (662, 2)


In [93]:
# Normalisation training:
scaler = MinMaxScaler(feature_range=(0, 1))
scaler = scaler.fit(train.iloc[:,1:])
print('Min: %f, Max: %f' % (scaler.data_min_, scaler.data_max_))
# Normalised dataset:
df_train = scaler.transform(train.iloc[:,1:])
df_test = scaler.transform(test.iloc[:,1:])

Min: 0.916750, Max: 8.916750


In [94]:
print(len(df_train), len(df_test))

1546 662


In [95]:
# Converting numpy array into matrix:
def matrix_dataset(array_dataset, timesteps=1):
 mat_X, mat_Y = [], []
 for i in range(len(array_dataset)-timesteps-1):
  Z = i+timesteps
  m = array_dataset[i:Z, 0]
  mat_X.append(m)
  n = array_dataset[Z, 0]
  mat_Y.append(n)
 return np.array(mat_X), np.array( mat_Y)

In [96]:
# Preparing the data and reshaping it as my  LSTM input layer must be 3D (2 dimensions - samples, time steps, and features)
timesteps =  10
train_X,train_y = matrix_dataset(df_train,timesteps)
test_X, test_y = matrix_dataset(df_test,timesteps)
print("The train_X and train_y values are:",train_X.shape,train_y.shape)
print("The test_X and test_y values are:",test_X.shape,test_y.shape)

The train_X and train_y values are: (1535, 10) (1535,)
The test_X and test_y values are: (651, 10) (651,)


In [128]:
mlp_model=Sequential()
mlp_model.add(Dense(128,input_dim=timesteps,activation='sigmoid'))
mlp_model.add(Dropout(0.3))
mlp_model.add(Dense(64,activation='sigmoid'))
mlp_model.add(Dropout(0.3))
mlp_model.add(Dense(32,activation='sigmoid'))
mlp_model.add(Dropout(0.3))
mlp_model.add(Dense(1,activation='linear'))
mlp_model.compile(loss='mean_squared_error',optimizer='adam')
mlp_model.fit(train_X,train_y,validation_data=(test_X,test_y),
              callbacks=[EarlyStopping(monitor='val_loss', patience=10)], verbose=1, shuffle=False,epochs=200,batch_size=200)

Epoch 1/200
8/8 [==============================] - 1s 28ms/step - loss: 0.4341 - val_loss: 0.0607
Epoch 2/200
8/8 [==============================] - 0s 8ms/step - loss: 0.3144 - val_loss: 0.0418
Epoch 3/200
8/8 [==============================] - 0s 7ms/step - loss: 0.2865 - val_loss: 0.0412
Epoch 4/200
8/8 [==============================] - 0s 8ms/step - loss: 0.2166 - val_loss: 0.0427
Epoch 5/200
8/8 [==============================] - 0s 8ms/step - loss: 0.1861 - val_loss: 0.0406
Epoch 6/200
8/8 [==============================] - 0s 7ms/step - loss: 0.1453 - val_loss: 0.0419
Epoch 7/200
8/8 [==============================] - 0s 8ms/step - loss: 0.1224 - val_loss: 0.0420
Epoch 8/200
8/8 [==============================] - 0s 8ms/step - loss: 0.1175 - val_loss: 0.0403
Epoch 9/200
8/8 [==============================] - 0s 8ms/step - loss: 0.1042 - val_loss: 0.0412
Epoch 10/200
8/8 [==============================] - 0s 31ms/step - loss: 0.0976 - val_loss: 0.0423
Epoch 11/200
8/8 [=========

In [124]:
mlp_model.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_39 (Dense)             (None, 128)               1408      
_________________________________________________________________
dropout_24 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_40 (Dense)             (None, 64)                8256      
_________________________________________________________________
dropout_25 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_41 (Dense)             (None, 32)                2080      
_________________________________________________________________
dropout_26 (Dropout)         (None, 32)                0         
_________________________________________________________________
dense_42 (Dense)             (None, 1)               

In [129]:
#Prediction and getting the accuracy metrics
predict_train=mlp_model.predict(train_X)
predict_test=mlp_model.predict(test_X)

##Transformback to original form
predict_train_mlp=scaler.inverse_transform(predict_train)
predict_test_mlp=scaler.inverse_transform(predict_test)

In [130]:
### Calculate RMSE performance metrics
import math
from sklearn.metrics import mean_squared_error
trainset = math.sqrt(mean_squared_error(train_y,predict_train_mlp))
testset = math.sqrt(mean_squared_error(test_y,predict_test_mlp))

print("The RMSE prediction value on trainset: ",trainset)
print("The RMSE prediction value on testset: ",testset)

The RMSE prediction value on trainset:  2.953188278709901
The RMSE prediction value on testset:  2.9419973069086613
